This exercise is from this oreilly book:   https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch11.html#object_repr_sec   

All the code is here: https://github.com/fluentpython/example-code-2e/tree/master/11-pythonic-obj

In [12]:
from array import array
import math

class Vector2d:
    # typecode is a class attribute we’ll use when converting 
    # Vector2d instances to/from bytes.

    typecode = 'd'

    # Converting x and y to float in __init__ catches errors 
    # early, which is helpful in case Vector2d is called with unsuitable arguments.
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    # This will make our Vector2d object hashable:
    # The @property decorator marks the getter method of a property.
    # Use exactly two leading underscores (with zero or one trailing 
    # underscore) to make an attribute private.  It’s not strictly necessary 
    # to implement properties or otherwise protect the instance attributes 
    # to create a hashable type. 
    # Implementing __hash__ and __eq__ correctly is all it takes. 

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __hash__(self):
        return hash((self.x), (self.y))

    # __iter__ makes a Vector2d iterable; this is what makes
    # unpacking work (e.g, x, y = my_vector). We implement it 
    # simply by using a generator expression to yield the 
    # components one after the other

    def __iter__(self):
        return(i for i in (self.x, self.y))
    

    
 # __repr__ builds a string by interpolating the components with {!r} 
 # to get their repr; because Vector2d is iterable, *self feeds the x 
 # and y components to format.
    
# The {!r} is a format string option that specifies that the value 
# should be displayed using its repr() representation. In Python, most 
# objects have both a str() representation and a repr() representation.

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
# From an iterable Vector2d, it’s easy to build a tuple for 
# display as an ordered pair.

    def __str__(self):
        return str(tuple(self))
    
# To generate bytes, we convert the typecode to bytes and concatenate…
# bytes converted from an array built by iterating over the instance.

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
# To quickly compare all components, build tuples out of the operands. 
# This works for operands that are instances of Vector2d, but has 
# issues. See the following warning.

    def __eq__(self, other):
        return math.hypot(self.x, self.y)

# The magnitude is the length of the hypotenuse of the right triangle 
# formed by the x and y components.

    def __abs__(self):
        return math.hypot(self.x, self.y)

# __bool__ uses abs(self) to compute the magnitude, then converts it to 
# bool, so 0.0 becomes False, nonzero is True.

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod                           #1
    def frombytes(cls, octets):               #2
        typecode = chr(octets[0])               #3
        memv = memoryview(octets[1:]).cast(typecode)    #4
        return cls(*memv)                     #5

    """
    1 The classmethod decorator modifies a method so it can be called directly on a class.
    You can then call the method on an instance of the method OR on the class itself.

    2 No self argument; instead, the class itself is passed as the first argument—conventionally named cls.

    3 Read the typecode from the first byte.

    4 Create a memoryview (see cells below for what a memoryview is) from the octets binary sequence and use the typecode to cast it.4

    5 Unpack the memoryview resulting from the cast into the pair of arguments needed for the constructor.
    """

    # The memoryview built in function lets you handle slices of arrays without copying bytes.
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)  #1
        return '({}, {})'.format(*components)           #2
    
    """
    1 Use the format built-in to apply the fmt_spec to each vector component, 
    building an iterable of formatted strings.

    2 Plug the formatted strings in the formula '(x, y)'.
    """

    # To do pattern matching you have to use a special class attribute 
    # (usually done at the top of the class), then you can
    # define the pattern matching function outside the class:

    __match_args__ = ('x', 'y')



### The entire class written ideally:   
To understand what's going on in the repr and format methods you will likely need to look here and learn about the Format String Mini Language: https://fpy.li/11-5 and https://fpy.li/fmtspec

In [ ]:
"""
A two-dimensional vector class

    >>> v1 = Vector2d(3, 4)
    >>> print(v1.x, v1.y)
    3.0 4.0
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector2d(3.0, 4.0)
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector2d(0, 0))
    (True, False)


Test of ``.frombytes()`` class method:

    >>> v1_clone = Vector2d.frombytes(bytes(v1))
    >>> v1_clone
    Vector2d(3.0, 4.0)
    >>> v1 == v1_clone
    True


Tests of ``format()`` with Cartesian coordinates:

    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'


Tests of the ``angle`` method::

    >>> Vector2d(0, 0).angle()
    0.0
    >>> Vector2d(1, 0).angle()
    0.0
    >>> epsilon = 10**-8
    >>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon
    True
    >>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon
    True


Tests of ``format()`` with polar coordinates:

    >>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector2d(1, 1), '.3ep')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector2d(1, 1), '0.5fp')
    '<1.41421, 0.78540>'


Tests of `x` and `y` read-only properties:

    >>> v1.x, v1.y
    (3.0, 4.0)
    >>> v1.x = 123
    Traceback (most recent call last):
      ...
    AttributeError: can't set attribute 'x'


Tests of hashing:

    >>> v1 = Vector2d(3, 4)
    >>> v2 = Vector2d(3.1, 4.2)
    >>> len({v1, v2})
    2

"""

from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [ ]:
def positional_pattern_demo(v: Vector)


In [13]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [14]:
x, y = v1
print(x, y)

3.0 4.0


In [15]:
v1

Vector2d(3.0, 4.0)

In [16]:
v1_clone = eval(repr(v1))
v1 == v1_clone

5.0

In [17]:
print(v1)

(3.0, 4.0)


In [18]:
octets = bytes(v1)
print(octets)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


In [19]:
abs(v1)

5.0

In [20]:
bool(v1), bool(Vector2d(0, 0))


(True, False)

A `memoryview` in Python is a built-in type that allows you to access and manipulate the internal data of an object without copying it. Essentially, it provides a way to interact with the memory of other data structures in a structured manner. This can be useful in scenarios where performance is a concern since it avoids unnecessary data copying.

Here are some key points about `memoryview`:

1. **Performance**: As mentioned, the primary reason for using a `memoryview` is performance. For example, if you have a large `bytearray` and you want to modify a chunk of its data, using a `memoryview` can let you do this without copying any data.

2. **Supports Slicing**: When you slice a `memoryview`, it doesn't create a copy. Instead, it returns another `memoryview` object that references the same memory.

3. **Diverse Types**: While `memoryview` is often used with `bytearray`, it can also work with other types like `array.array`.

4. **Format**: `memoryview` objects have a format that specifies the kind of data (e.g., bytes, integers, floats) they are referencing.

5. **Multidimensional Arrays**: Beyond just 1D sequences, `memoryview` can handle multi-dimensional data with the concept of strides.

Here's a simple example:

```python
# Create a bytearray
data = bytearray(b'abcdefgh')

# Create a memoryview of the bytearray
mv = memoryview(data)

# Let's change the second byte (in-place) using the memoryview
mv[1] = ord('B')

# Check the original bytearray
print(data)  # Outputs: bytearray(b'aBcdefgh')
```

In the example above, we used a `memoryview` to modify the contents of a `bytearray` without ever creating a copy of the data.

In short, `memoryview` offers a way to handle memory more efficiently, especially when dealing with large data or when performing operations that would otherwise involve copying data.